# Script to analyse UKBB shole brain data in the obesity accross lifespan (OBAL) project

## Load packages

In [1]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)

## Load the data

In [53]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])

ukbb_check=merge(my_ukb_data,di, by.x='eid')

di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Schizophrenia
di$schizophrenia <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$schizophrenia[di[i]=="Schizophrenia"] <- 1
}


# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i]=="Heart attack"] <-1
di$vascular_heart_diagnoses[di[i] =="Angina"] <- 1
di$vascular_heart_diagnoses[di[i] =="Stroke"] <- 1
}


#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
di$excluded[di$bipolar_dis ==1] <- 1
di$excluded[di$personality_dis == 1] <- 1
di$excluded[di$schizophrenia == 1] <- 1
di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')

nrow(ukbb_all)

## Remove outliers

In [52]:
grep('volume', colnames(ukbb_all))

In [54]:
variables_for_outliers=c(58:88, 120:150, 26:27, 32:38, 42:43, 48:54, 89:119, 151:181, 1891, 9) # all volume and thickness variables from FS, BMI, eTIV 

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_all[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_all[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_all[j][ukbb_all[j]<lower]=NA
    ukbb_all[j][ukbb_all[j]>upper]=NA
}

ukbb_all=ukbb_all[complete.cases(ukbb_all[,variables_for_outliers]),]
nrow(ukbb_all)

## Save file with outliers and clinical groups excluded

In [55]:
write.table(ukbb_all, file = "/dagher/dagher11/filip/OBAL/output/UKBB_preprocessed.csv", 
            append = FALSE, quote = FALSE, sep = ",",
            eol = "\n", na = "NA", dec = ".", row.names = FALSE,
            col.names = TRUE)

## Calculate BMI * Age interaction influence on cortical thickness

In [56]:
Stats=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]])-mean(ukbb_all[[i]]))/sd(ukbb_all[[i]])
    
    a=(summary(lmp(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    Stats[index,][2]=a$coefficients[6,][3]
    Stats[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

Stats_t=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]])-mean(ukbb_all[[i]]))/sd(ukbb_all[[i]])
    
    a=(summary(lm(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]))))
    
    Stats_t[index,][2]=a$coefficients[6,][3]
    Stats_t[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

## Use BH FDR correction for the p-values and save them to be plotted in MATLAB

In [73]:
write.table(p.adjust(Stats[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrain_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Stats_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrain_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## Age * BMI for subcortical volumes

In [58]:
Statssubvol=matrix(nrow=length(c(26:27,32:38, 42:43, 48:54)),ncol=2)
index=1
for (i in c(26:27,32:38, 42:43, 48:54)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]]/ukbb_all[[9]])-mean(ukbb_all[[i]]/ukbb_all[[9]]))/sd(ukbb_all[[i]]/ukbb_all[[9]])
    
    b=(summary(lmp(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    Statssubvol[index,][2]=b$coefficients[5,][3]
    Statssubvol[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

Statssubvol_t=matrix(nrow=length(c(26:27,32:38, 42:43, 48:54)),ncol=2)
index=1
for (i in c(26:27,32:38, 42:43, 48:54)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]])-mean(ukbb_all[[i]]))/sd(ukbb_all[[i]])
    
    b=(summary(lm(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]))))
    
    Statssubvol_t[index,][2]=b$coefficients[5,][3]
    Statssubvol_t[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

In [75]:
print(Statssubvol_t[p.adjust(Statssubvol[,2], method = 'BH')<0.05])

In [74]:
write.table(p.adjust(Statssubvol[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainvol_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Statssubvol_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainvol_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## Age * BMI for volume of cortical gray matter

In [60]:
Statsctxvol=matrix(nrow=length(c(89:119, 151:181)),ncol=2)
index=1
for (i in c(89:119, 151:181)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]]/ukbb_all[[9]])-mean(ukbb_all[[i]]/ukbb_all[[9]]))/sd(ukbb_all[[i]]/ukbb_all[[9]])
    
    c=(summary(lmp(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    Statsctxvol[index,][2]=c$coefficients[5,][3]
    Statsctxvol[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

Statsctxvol_t=matrix(nrow=length(c(89:119, 151:181)),ncol=2)
index=1
for (i in c(89:119, 151:181)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]]/ukbb_all[[9]])-mean(ukbb_all[[i]]/ukbb_all[[9]]))/sd(ukbb_all[[i]]/ukbb_all[[9]])
    
    c=(summary(lm(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]))))
    
    Statsctxvol_t[index,][2]=c$coefficients[5,][3]
    Statsctxvol_t[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

In [76]:
Statsctxvol_t[,2][p.adjust(Statsctxvol[,2], method = 'BH')<0.05]

In [77]:
write.table(p.adjust(Statsctxvol[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainvolctx_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Statsctxvol_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainvolctx_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## Thickness and BMI

In [62]:
Statsctbmi=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]])-mean(ukbb_all[[i]]))/sd(ukbb_all[[i]])
    
    d=(summary(lmp(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    Statsctbmi[index,][2]=d$coefficients[2,][3]
    Statsctbmi[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

Statsctbmi_t=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]])-mean(ukbb_all[[i]]))/sd(ukbb_all[[i]])
    
    d=(summary(lm(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]))))
    
    Statsctbmi_t[index,][2]=d$coefficients[2,][3]
    Statsctbmi_t[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

In [86]:
print(Statsctbmi_t[,2][p.adjust(Statsctbmi[,2], method = 'BH')<0.05])
print(Statsctbmi_t[,1][p.adjust(Statsctbmi[,2], method = 'BH')<0.05])

In [79]:
write.table(p.adjust(Statsctbmi[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainBMI_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Statsctbmi_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainBMI_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## BMI * Age for surface area (calculated from volume and thickness) - WRONG

In [29]:
Statsarea=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i+31]]/ukbb_all[[i]])-mean(ukbb_all[[i+31]]/ukbb_all[[i]]))/sd(ukbb_all[[i+31]]/ukbb_all[[i]])
    
    e=(summary(lmp(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    Statsarea[index,][2]=e$coefficients[2,][3]
    Statsarea[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

Statsarea_t=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i+31]]/ukbb_all[[i]])-mean(ukbb_all[[i+31]]/ukbb_all[[i]]))/sd(ukbb_all[[i+31]]/ukbb_all[[i]])
    
    e=(summary(lm(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]))))
    
    Statsarea_t[index,][2]=e$coefficients[2,][3]
    Statsarea_t[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

In [328]:
Statsarea_t[,2][p.adjust(Statsarea[,2], method = 'BH')<0.05]

In [ ]:
write.table(p.adjust(Statsarea[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainarea_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Statsarea_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainarea_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## Sanity check - cortical thickness and age

In [98]:
Statsctage=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]])-mean(ukbb_all[[i]]))/sd(ukbb_all[[i]])
    
    d=(summary(lmp(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    Statsctage[index,][2]=d$coefficients[3,][3]
    Statsctage[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

Statsctage_t=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_all[[i]])-mean(ukbb_all[[i]]))/sd(ukbb_all[[i]])
    
    d=(summary(lm(scaled ~ 
                     scale(log(ukbb_all[[1891]]))*
                     scale(ukbb_all[[1897]]) + 
                     as.factor(ukbb_all[[185]]) +
                     scale(ukbb_all[[9]]))))
    
    Statsctage_t[index,][2]=d$coefficients[3,][3]
    Statsctage_t[index,][1]=colnames(ukbb_all)[i]
    index=index+1
    
}

In [99]:
write.table(p.adjust(Statsctage[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainctage_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Statsctage_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainctage_tval.csv',sep=',',quote=FALSE, row.names=FALSE)